# Assessment of the performance of the RA and PC-TSP models
## Based on computational time on various datasets


### Plan:

- Load all functions from modules, all dependencies
- Load all datasets for the analysis
- Run each algorithm in a loop, saving the time of ONLY the optimization algorithm
- Save the timing into a df
- Show the timings in a table (with the dataset details, objective values, time taken by each algorithm)
- Turn the table into a latex code for the report

In [1]:
from Data import *
from pctstp import *
from tiaor import *
import os

In [2]:
# List the .xlsx files in the Data folder
files = os.listdir("Data")
filepaths = ["data/" + file for file in files if file.endswith('.xlsx')]
print(filepaths)

# Make dictionaries to store the data for each file
ra_data = {}
pctsp_data = {}

# Load the data from each file
for filepath in filepaths:
    file_name = os.path.basename(filepath)
    file_name = file_name.split('.')[0]
    pairs, altruistic_donors, nodes, edges = get_data(filepath, pc_tsp=False)
    ra_data[file_name] = {
        "pairs": pairs,
        "altruistic_donors": altruistic_donors,
        "nodes": nodes,
        "edges": edges
    }
    G_pctsp, pairs_pctsp, altruistic_donors_pctsp, nodes_pctsp, edges_pctsp, all_cycles_pctsp = get_data(filepath, max_cycle=3, pc_tsp=True)
    pctsp_data[file_name] = {
        "G": G_pctsp,
        "pairs": pairs_pctsp,
        "altruistic_donors": altruistic_donors_pctsp,
        "nodes": nodes_pctsp,
        "edges": edges_pctsp,
        "all_cycles": all_cycles_pctsp
    }


['data/Dataset1.xlsx', 'data/Dataset10.xlsx', 'data/Dataset11.xlsx', 'data/Dataset12.xlsx', 'data/Dataset13.xlsx', 'data/Dataset14.xlsx', 'data/Dataset15.xlsx', 'data/Dataset16.xlsx', 'data/Dataset17.xlsx', 'data/Dataset18.xlsx', 'data/Dataset19.xlsx', 'data/Dataset2.xlsx', 'data/Dataset20.xlsx', 'data/Dataset21.xlsx', 'data/Dataset22.xlsx', 'data/Dataset23.xlsx', 'data/Dataset24.xlsx', 'data/Dataset25.xlsx', 'data/Dataset3.xlsx', 'data/Dataset4.xlsx', 'data/Dataset5.xlsx', 'data/Dataset6.xlsx', 'data/Dataset7.xlsx', 'data/Dataset8.xlsx', 'data/Dataset9.xlsx']


In [3]:
# Example of accessing the data
check_dataset = pctsp_data['Dataset1']
print(check_dataset['all_cycles'])


{(0.0, 189.0, 93.0): 89.0, (0.0, 217.0, 65.0): 132.0, (2.0, 22.0, 167.0): 150.0, (2.0, 22.0, 217.0): 192.0, (2.0, 25.0, 207.0): 145.0, (2.0, 76.0, 5.0): 153.0, (2.0, 84.0, 5.0): 187.0, (2.0, 193.0, 15.0): 188.0, (2.0, 193.0, 126.0): 205.0, (2.0, 193.0, 155.0): 134.0, (2.0, 193.0, 167.0): 224.0, (2.0, 214.0, 93.0): 126.0, (3.0, 179.0, 160.0): 203.0, (4.0, 179.0, 220.0): 171.0, (5.0, 7.0, 76.0): 91.0, (5.0, 15.0, 76.0): 93.0, (5.0, 15.0, 84.0): 106.0, (5.0, 18.0, 76.0): 131.0, (5.0, 20.0, 76.0): 120.0, (5.0, 22.0, 76.0): 181.0, (5.0, 33.0, 76.0): 49.0, (5.0, 51.0, 76.0): 100.0, (5.0, 61.0, 76.0): 172.0, (5.0, 66.0, 76.0): 132.0, (5.0, 77.0, 76.0): 213.0, (5.0, 80.0, 76.0): 147.0, (5.0, 83.0, 76.0): 162.0, (5.0, 91.0, 76.0): 163.0, (5.0, 94.0, 76.0): 106.0, (5.0, 117.0, 76.0): 175.0, (5.0, 132.0, 76.0): 210.0, (5.0, 136.0, 15.0): 188.0, (5.0, 136.0, 76.0): 152.0, (5.0, 136.0, 190.0): 253.0, (5.0, 139.0, 76.0): 63.0, (5.0, 169.0, 76.0): 147.0, (5.0, 171.0, 76.0): 140.0, (5.0, 174.0, 76.0):

In [4]:
# Load existing results if they exist
results_file = 'results_natalia.csv'
if os.path.exists(results_file):
    results = pd.read_csv(results_file)
else:
    results = pd.DataFrame(columns = ['Dataset', 'NDDs', 'Pairs', 'Edges', 'Cycles', 'RA', 'PCTSP', 'Objective Value RA', 'Objective Value PCTSP'])
    results['Dataset'] = [os.path.basename(filepath).split('.')[0] for filepath in filepaths]
    results['NDDs'] = [len(ra_data[dataset]['altruistic_donors']) for dataset in ra_data]
    results['Pairs'] = [len(ra_data[dataset]['pairs']) for dataset in ra_data]
    results['Edges'] = [len(ra_data[dataset]['edges']) for dataset in ra_data]
    results['Cycles'] = [len(pctsp_data[dataset]['all_cycles']) for dataset in pctsp_data]
    results.to_csv(results_file, index=False)

# Run the RA and PCTSP algorithms on each dataset and save the results
for dataset in ra_data:
    # # I realise this is a very badly written line, but I was adding the failed datasets as they came
    # if dataset == 'Dataset16' or dataset == 'Dataset17' or dataset == 'Dataset18' or dataset == 'Dataset20' or dataset == "Dataset22" or dataset == "Dataset25" or dataset == "Dataset3" or dataset == "Dataset5" or dataset=="Dataset7" or dataset=="Dataset9":
    #     continue
    
    if dataset in results['Dataset'].values and results.loc[results['Dataset'] == dataset, ['RA', 'PCTSP']].notnull().all(axis=None).all():
        print(f"Skipping {dataset} as it has already been processed.")
        continue
    
    pairs = ra_data[dataset]['pairs']
    altruistic_donors = ra_data[dataset]['altruistic_donors']
    nodes = ra_data[dataset]['nodes']
    edges = ra_data[dataset]['edges']
    
    G_pctsp = pctsp_data[dataset]['G']
    pairs_pctsp = pctsp_data[dataset]['pairs']
    altruistic_donors_pctsp = pctsp_data[dataset]['altruistic_donors']
    nodes_pctsp = pctsp_data[dataset]['nodes']
    edges_pctsp = pctsp_data[dataset]['edges']
    all_cycles_pctsp = pctsp_data[dataset]['all_cycles']
    
    print(f"Running on {dataset}")
    opt_val_ra, _, time_taken_ra, *rest = ra(pairs, altruistic_donors, edges, noisy=0)
    opt_val_pctsp, _, _, time_taken_pctsp = pctsp(G_pctsp, pairs_pctsp, altruistic_donors_pctsp, nodes_pctsp, edges_pctsp, all_cycles_pctsp, noisy=0)

    results.loc[results['Dataset'] == dataset, ['RA', 'PCTSP', 'Objective Value RA', 'Objective Value PCTSP']] = [time_taken_ra, time_taken_pctsp, opt_val_ra, opt_val_pctsp]

# Save the updated results to the CSV file
results.to_csv(results_file, index=False)

Skipping Dataset1 as it has already been processed.
Skipping Dataset10 as it has already been processed.
Skipping Dataset11 as it has already been processed.
Skipping Dataset12 as it has already been processed.
Skipping Dataset13 as it has already been processed.
Skipping Dataset14 as it has already been processed.
Skipping Dataset15 as it has already been processed.
Skipping Dataset16 as it has already been processed.
Skipping Dataset17 as it has already been processed.
Skipping Dataset18 as it has already been processed.
Skipping Dataset19 as it has already been processed.
Skipping Dataset2 as it has already been processed.
Skipping Dataset20 as it has already been processed.
Skipping Dataset21 as it has already been processed.
Skipping Dataset22 as it has already been processed.
Skipping Dataset23 as it has already been processed.
Skipping Dataset24 as it has already been processed.
Skipping Dataset25 as it has already been processed.
Running on Dataset3


c:\Users\s1975950\Desktop\taor_project\tiaor.py:48: LicenseWarning: Using the license file found in your Xpress installation. If you want to use this license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('C:/xpressmp/bin/xpauth.xpr')

  prob = xp.problem()


Running on Dataset4
Running on Dataset5
Running on Dataset6
Running on Dataset7
Running on Dataset8
Running on Dataset9


In [6]:
# Run this if you get new results from the above cell
# Save the updated results to the CSV file
results.to_csv(results_file, index=False)